In [5]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [6]:
from openai_hackathon_wait.tools.pubmed_tool import pubmed_tool

In [ ]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

# arxiv_agent = Agent(
#     name="Arxiv agent",
#     instructions="You provide information about the papers that are related to the query.",
#     tools=[arxiv_search],
#     model="gpt-4o-mini",
# )

pubmed_agent = Agent(
    name="Pubmed agent",
    instructions="""You are a medical research assistant specializing in finding and summarizing
    relevant scientific literature from PubMed. Use the search_pubmed tool to find information
    about medical topics, diseases, treatments, and research.
    
    When searching PubMed:
    1. Formulate specific search queries to get precise results
    2. Extract key findings and insights from the articles
    3. Present the information in a clear, structured format
    4. Cite the sources properly
    5. Be honest when information is not found or unclear
    """,
    tools=[pubmed_tool],
    model="gpt-4o-mini",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="You are a helpful assistant that finds the most relevant papers from tools that have access to the databases.",
    handoffs=[pubmed_agent],
    model="gpt-4o-mini",
)

async def main():
    result = await Runner.run(
        triage_agent, input="Find papers about rheumatoid arthritis"
    )
    print(result.final_output)

await main()


Searching PubMed for: rheumatoid arthritis
Searching PubMed for: rheumatoid arthritis treatment
Here are some recent papers related to rheumatoid arthritis:

1. **Association Between T2DM, TyG Index, Multiple Sclerosis, and Rheumatoid Arthritis: Insights into Potential Mechanisms**
   - **Published:** April 25, 2025
   - **Summary:** This study examines the causal relationships between type 2 diabetes mellitus (T2DM), the triglyceride-glucose (TyG) index, and autoimmune-related diseases, including rheumatoid arthritis (RA). It utilizes Mendelian randomization to analyze data from genome-wide association studies and identifies immune cell involvement and genetic variations impacting these associations.
   - **Link:** [Read Article](https://example.com)

2. **Patient-Reported Outcome Measures in Patients with Rheumatoid Arthritis, Psoriasis, or Psoriatic Arthritis Treated with GP2015, an Etanercept Biosimilar: Results from Two Phase III Studies (EGALITY and EQUIRA)**
   - **Published:** 